# Identify the differences and advantages of model serving approaches: batch, realtime, and streaming


**Batch (lote):** Ejecuta predicciones sobre un conjunto grande de registros según un horario o un disparador (por ejemplo, cada hora, cada noche o bajo demanda). Los resultados se almacenan (tabla/archivo) y se consumen después.

**Tiempo real (online):** Sirve predicciones por solicitud a través de un endpoint API; devuelve la respuesta en milisegundos o pocos segundos.

**Streaming:** Puntúa datos continuos y sin fin (por ejemplo, Kafka o Auto Loader hacia Delta) con latencia baja y constante; los resultados se envían de forma continua a un destino.

**Comparativa rápida**

| Dimensión             | Batch                                                          | Tiempo real                                      | Streaming                                               |
| --------------------- | -------------------------------------------------------------- | ------------------------------------------------ | ------------------------------------------------------- |
| **Disparador**        | Programado / job bajo demanda                                  | Solicitud HTTP/gRPC                              | Llega un nuevo evento                                   |
| **Latencia objetivo** | Minutos → horas (a veces segundos)                             | \~5–3000 ms                                      | Subsegundo → segundos, continuo                         |
| **Patrón de volumen** | Muy alto, en oleadas                                           | Por solicitud; tráfico irregular                 | Flujo constante alto                                    |
| **Estado y orden**    | Instantánea o micro-batches                                    | Generalmente sin estado                          | Puede ser con estado (ventanas, joins)                  |
| **Costo**             | Bajo costo por predicción en grandes volúmenes                 | Más caro por estar siempre activo                | Costo estable para cargas constantes                    |
| **Manejo de fallos**  | Fácil reintento; reprocesos idempotentes                       | Retrys en cliente/gateway; cuidado con timeouts  | Checkpoints, garantías “exactly-once” o “at-least-once” |
| **Mejor para**        | Backfills, puntajes nocturnos, features para BI, listas de CRM | Personalización UX, fraude en checkout, chatbots | Clickstream, IoT, detección de anomalías                |
| **Anti-patrón común** | Querer ms de latencia usando batch                             | Usar online para millones de registros           | Usar streaming cuando batch cada hora es suficiente     |


## 📌 Puntos clave
**Tres reglas rápidas para decidir**

**¿Quién espera?**
Si alguien o un sistema espera la respuesta → Tiempo real. Si no → considera Batch o Streaming.

¿Cómo llega el dato?
En bloques → Batch. Flujo continuo → Streaming.

¿Frescura vs costo?
Si “minutos” es aceptable para grandes volúmenes → Batch.
Si necesitas segundos y actualización constante → Streaming.
Si necesitas <1 segundo para un registro → Tiempo real.

**En Databricks (enfoque del examen)**

Batch: Jobs/Workflows que leen de Delta, puntúan con pandas o Spark UDFs, y escriben resultados a Delta. Usado para backfills y scoring periódico.

Tiempo real: Model Serving endpoints (MLflow). Reciben peticiones REST, permiten canary y división de tráfico, con autoscaling y autenticación.

Streaming: Delta Live Tables (DLT) o Structured Streaming para leer de Auto Loader/Kafka, generar features (ventanas/joins), puntuar modelos (UDFs o endpoints externos) y guardar a Delta o buses de eventos. Importante: checkpoints y watermarks.

# Deploy a custom model to a model endpoint

**1. ¿Qué significa “custom model”?**

En Databricks puedes desplegar modelos entrenados con frameworks conocidos (sklearn, XGBoost, PyTorch, TensorFlow).
Un modelo personalizado significa que tú defines la lógica de inferencia en lugar de usar el flavor automático de MLflow.

Ejemplo: quieres preprocesar texto antes de predecir, o devolver múltiples salidas, o integrar librerías externas.

En este caso, defines tu propio pyfunc model o escribes la lógica de predict().

**2. Flujo general en Databricks**

**1.1 Entrenar y guardar el modelo**

- Entrenas en Databricks Notebook / Jobs.
- Guardas el modelo con MLflow:

In [0]:
import mlflow.pyfunc

class MyCustomModel(mlflow.pyfunc.PythonModel):
    def predict(self, context, model_input):
        # lógica de preprocesamiento + inferencia
        # ej: limpiar texto antes de usar un clasificador
        processed = model_input.str.lower()
        return ["OK" if "good" in x else "BAD" for x in processed]

mlflow.pyfunc.save_model(path="custom_model", python_model=MyCustomModel())


**1.2 Registrar el modelo en Unity Catalog**

Usas MLflow para registrar el modelo, Así el modelo queda versionado y gobernado.:

In [0]:
mlflow.register_model("runs:/<run-id>/model", "catalog.schema.custom_model")

**1.3 Crear un endpoint de Serving**

Desde la UI:

- Machine Learning → Model Serving → Create Serving Endpoint
- Seleccionas el modelo registrado y la versión.

O desde API/CLI:

In [0]:
import databricks.sdk.service.serving as serving

client = DatabricksClient()
client.serving_endpoints.create(
    name="custom-model-endpoint",
    config=serving.EndpointCoreConfigInput(
        served_models=[serving.ServedModelInput(
            name="custom-model",
            model_name="catalog.schema.custom_model",
            model_version="1",
            workload_size="Small"
        )]
    )
)


**Diferencia entre Deploy con flavor y Deploy custom**

**1) Flavors (sabores MLflow)**

Qué es: MLflow ya trae integraciones (flavors) para frameworks comunes:

- mlflow.sklearn
- mlflow.xgboost
- mlflow.pytorch
- mlflow.tensorflow
- mlflow.lightgbm, etc.

**Cómo funciona:**

- Guardas el modelo con un flavor:

```
import mlflow.sklearn
mlflow.sklearn.log_model(model, "model")
```

- MLflow sabe automáticamente cómo serializar, cargar y servir ese modelo.
- El endpoint desplegado entiende la API estándar de predicción (predict()) de ese framework.

**Ventajas:**

- Simplicidad → rápido y sin código extra.
- Compatibilidad → Databricks Serving puede levantarlo sin problemas.
- Ideal cuando no necesitas lógica extra, solo correr el modelo entrenado.

**Limitación:**

- No puedes personalizar la lógica de predicción más allá de lo que hace el framework.
- No es flexible para pipelines complejos, preprocesamiento/postprocesamiento.

**2) Custom model (PythonModel / pyfunc)**

Qué es: Definido por ti mediante la interfaz mlflow.pyfunc.PythonModel.

**Cómo funciona:**

```
import mlflow.pyfunc

class MyCustomModel(mlflow.pyfunc.PythonModel):
    def predict(self, context, model_input):
        # lógica definida por ti
        # puedes incluir preprocesamiento + modelo + postprocesamiento
        return ["OK" if "good" in x else "BAD" for x in model_input]

mlflow.pyfunc.save_model(path="custom_model", python_model=MyCustomModel())
```

**Ventajas:**

- Flexibilidad total → defines cómo procesar la entrada y la salida.
- Puedes combinar varios modelos (ensembles).
- Puedes añadir transformaciones personalizadas (ej. NLP, normalización).
- Permite integrar librerías externas que no tienen flavor oficial.

**Limitación:**

- Más código a mantener.
- Mayor riesgo de errores si no manejas bien dependencias o entornos.
- Requiere empaquetar dependencias (conda.yaml, requirements.txt).

## 📌 Puntos clave
**Beneficios de usar endpoints en Databricks**

- Escalado automático: ajusta recursos según tráfico.
- Seguridad: integración con Unity Catalog y permisos RBAC.
- Observabilidad: métricas de latencia (p50/p95/p99), throughput, logs de errores.
- Integración con CI/CD: cambiar versiones con aliases (champion, challenger).
- Customización: lógica de inferencia avanzada, ensembles, post-procesamiento.

---
**Flavor =** rápido, estándar, soportado directamente por MLflow.

**Custom =** cuando necesitas más control (pre/post procesamiento, ensembles, frameworks no soportados).

---
✅ En el examen:

- Tienes que distinguir qué significa “deploy custom model” → registrar con MLflow, crear endpoint, y exponer lógica propia.
- Reconocer ventajas: gobernanza, escalado, observabilidad, seguridad.
- Identificar cuándo usar endpoint vs batch.

# Use pandas to perform batch inference

**1. Concepto básico**

**Batch inference =** aplicar un modelo a un conjunto de datos ya almacenado (ej. Delta table, CSV, Parquet) en lugar de responder peticiones en tiempo real.

El flujo es:

- Cargar modelo desde el registro (mlflow.pyfunc.load_model o mlflow.sklearn.load_model).
- Cargar datos a un DataFrame de pandas.
- Usar .predict() para obtener resultados.
- Guardar los resultados en una tabla, archivo o endpoint downstream.

**2. Ejemplo práctico**

Supongamos que registraste un modelo de clasificación en MLflow con alias "champion":

In [0]:
import mlflow
import pandas as pd

# 1. Cargar modelo registrado
model_uri = "models:/my_model@champion"
model = mlflow.pyfunc.load_model(model_uri)

# 2. Cargar datos batch (ejemplo CSV o Delta)
df = pd.read_csv("/dbfs/FileStore/batch_input.csv")

# 3. Inference con pandas DataFrame
preds = model.predict(df)

# 4. Guardar resultados
output = df.copy()
output["prediction"] = preds
output.to_csv("/dbfs/FileStore/batch_output.csv", index=False)


## 📌 Puntos clave
Entrada esperada: el modelo sirve datos en forma de pandas.DataFrame.

**Salida esperada:** depende del modelo, pero usualmente es numpy.ndarray, pandas.Series, o un DataFrame con columnas predichas.
**Escalabilidad:** aunque usas pandas localmente, en Databricks puedes escalar con Spark → convertir spark.DataFrame a pandas.DataFrame para lotes pequeños, o usar applyInPandas() para procesar en paralelo lotes grandes.

Casos de uso: predicciones nocturnas, scoring de millones de filas, ETL con features ya preparados.

**Diferencia con Real-time**

**Batch:** procesas todo un dataset a la vez (ejemplo: predecir churn para 10M de clientes esta noche).
**Real-time:** procesas una fila o request a la vez (ejemplo: recomendar productos cuando un cliente entra al sitio).

# Identify how streaming inference is performed with Delta Live Tables

**1. Concepto**

- Streaming inference = aplicar un modelo de ML continuamente sobre datos que llegan en tiempo real (por ejemplo, eventos de IoT, logs, clics de usuarios).
- En Databricks, se implementa con Delta Live Tables (DLT) porque estas permiten definir pipelines declarativos que soportan tanto batch como streaming.
- 
**2. Flujo típico con DLT**

- Fuente de datos (ejemplo: Kafka, Event Hub, Kinesis) → ingestada como streaming DataFrame.
- Transformación en DLT → procesamiento declarativo con funciones de limpieza, enriquecimiento y preparación de features.
- Inference con modelo MLflow → usar el modelo registrado (mlflow.pyfunc.spark_udf) como una UDF en el flujo.
- Salida → escribir resultados en una tabla Delta (batch + streaming compatible), dashboards en tiempo real o endpoints downstream.

In [0]:
import dlt
import mlflow

# 1. Cargar modelo registrado como UDF Spark
model_uri = "models:/fraud_detector@champion"
predict_udf = mlflow.pyfunc.spark_udf(spark, model_uri, result_type="double")

# 2. Definir tabla streaming en DLT
@dlt.table
def scored_transactions():
    return (
        spark.readStream.format("delta").table("raw_transactions")
        .withColumn("fraud_score", predict_udf("feature1", "feature2", "feature3"))
    )


## 📌 Puntos clave
- **DLT + MLflow: **el modelo se convierte en una UDF con mlflow.pyfunc.spark_udf.
- **Streaming:** readStream y writeStream son compatibles dentro de DLT.
- **Fiabilidad: **DLT maneja checkpointing y retries automáticamente → garantiza tolerancia a fallos.
- **Exactly-once processing:** DLT asegura que cada evento se procese una sola vez en caso de fallos, lo cual es crítico para inferencia.

**Casos de uso:**

- Detección de fraude en transacciones.
- Alertas en IoT en tiempo real.
- Recomendaciones instantáneas en e-commerce.

#  Deploy and query a model for realtime inference

**1. Concepto**

Realtime inference = publicar un modelo como un endpoint REST que puede responder a predicciones inmediatas para entradas individuales o en lotes pequeños (milisegundos–segundos).

En Databricks esto se hace con Model Serving, integrado con Unity Catalog y MLflow.

**2. Flujo típico**

- Registrar el modelo en Unity Catalog (con MLflow → flavors o pyfunc).
- Crear el endpoint de serving desde la UI o vía API:
- Se define un alias (champion, challenger) para controlar la versión servida.
- Se pueden asociar varios modelos al mismo endpoint con traffic splitting.
- Enviar solicitudes REST al endpoint para obtener predicciones.
- Se usa curl, requests en Python, o librerías cliente.

**3. Ejemplo – Deploy**

In [0]:
# a) Registro del modelo

import mlflow

# Entrenar o cargar un modelo sklearn
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

with mlflow.start_run():
    mlflow.sklearn.log_model(model, "model", registered_model_name="catalog.schema.my_model")


In [0]:
# b) Crear el endpoint (UI o API)
databricks model-serving-endpoints create \
  --name my-endpoint \
  --config '{
    "served_models": [{
      "model_name": "catalog.schema.my_model",
      "model_version": "1",
      "workload_size": "Small"
    }]
  }'


In [0]:
# 4. Ejemplo – Query

# a) Enviar solicitud REST (JSON)
curl -X POST https://<workspace-url>/serving-endpoints/my-endpoint/invocations \
  -H "Authorization: Bearer $DATABRICKS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{"dataframe_split": {"columns": ["feature1","feature2"], "data": [[1.0, 3.5]]}}'

# Python con requests
import requests, os, json

url = "https://<workspace-url>/serving-endpoints/my-endpoint/invocations"
headers = {"Authorization": f"Bearer {os.environ['DATABRICKS_TOKEN']}", "Content-Type": "application/json"}

payload = {"dataframe_split": {"columns": ["feature1","feature2"], "data": [[1.0, 3.5]]}}
response = requests.post(url, headers=headers, data=json.dumps(payload))

print(response.json())



## 📌 Puntos clave

Características importantes para el examen

- SLA: Model Serving en Databricks ofrece latency SLA para workloads pro (p95 < 100 ms).
- Escalado automático: endpoints se escalan según tráfico.
- Versioning con aliases: @champion y @challenger facilitan A/B testing.
- Traffic splitting: se puede dividir tráfico entre modelos (ej. 90% vs 10%).
- Compatibilidad: soporta sklearn, xgboost, pytorch, transformers y custom pyfunc.

| **Aspecto**      | **Batch inference (pandas/Spark)** | **Realtime inference (endpoints)** | **Streaming inference (DLT)** |
| ---------------- | ---------------------------------- | ---------------------------------- | ----------------------------- |
| Latencia         | Minutos–horas                      | Milisegundos–segundos              | Segundos–subsegundos          |
| Escenario típico | Scoring histórico (archivos)       | API en vivo (web/app)              | Flujo continuo (IoT, eventos) |
| Herramienta      | Pandas/Spark                       | Model Serving (REST endpoint)      | DLT + MLflow UDF              |


# Split data between endpoints for realtime interference

**1. Concepto**

- En Databricks no necesitas crear múltiples endpoints para probar distintos modelos.
- Un solo endpoint puede servir varios modelos.
- Puedes dividir el tráfico (split) entre ellos en porcentajes (ej. 90% Champion, 10% Challenger).

Esto se usa para:

- A/B testing: comparar dos modelos en producción.
- Canary deployment: liberar gradualmente un modelo nuevo antes de reemplazar al anterior.
- Experimentación segura: evaluar precisión y latencia en vivo con usuarios reales.

**2. Cómo funciona**
Cada served_model dentro del endpoint tiene:

- model_name
- model_version
- traffic_percentage

El endpoint automáticamente enruta solicitudes siguiendo esos porcentajes.

**3. Ejemplo – Configuración**

Crear un endpoint con 2 modelos (Esto envía el 80% de solicitudes al modelo v3 (champion) y el 20% al modelo v4 (challenger).)

In [0]:
databricks model-serving-endpoints create \
  --name my-endpoint \
  --config '{
    "served_models": [
      {
        "model_name": "catalog.schema.my_model",
        "model_version": "3",
        "workload_size": "Small",
        "traffic_percentage": 80
      },
      {
        "model_name": "catalog.schema.my_model",
        "model_version": "4",
        "workload_size": "Small",
        "traffic_percentage": 20
      }
    ]
  }'


In [0]:
### Actualizar el split (promoción progresiva) Aquí promovemos el modelo v4 a 100% tráfico, descartando v3.
databricks model-serving-endpoints update \
  --name my-endpoint \
  --config '{
    "served_models": [
      {
        "model_name": "catalog.schema.my_model",
        "model_version": "4",
        "workload_size": "Small",
        "traffic_percentage": 100
      }
    ]
  }'


## 📌 Puntos clave

Ventajas

✅ Sin downtime: no necesitas detener el endpoint para cambiar modelo.
✅ Promoción gradual: reduces riesgo al introducir un modelo nuevo.
✅ Experimentos controlados: puedes evaluar métricas de latencia (p95/p99) y precisión real.
✅ Rollback sencillo: si el Challenger falla, solo vuelves a aumentar el porcentaje del Champion.

| **Estrategia**               | **Uso principal**                                      |
| ---------------------------- | ------------------------------------------------------ |
| **Champion/Challenger**      | Probar un nuevo modelo contra el actual en paralelo.   |
| **Traffic Split (A/B test)** | Evaluar rendimiento en producción con usuarios reales. |
| **Canary Deployment**        | Migrar progresivamente (ej. 5% → 20% → 100%).          |

